In [0]:
# Upload File
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
# Read data content
f =open("/content/wdbc.data", "r")
if f.mode == 'r':
    dataContents =f.readlines()

dataContents

In [0]:
# Creating list of list of strings for data content
splitedContents = [i.split(",") for i in dataContents]
splitedContents

In [0]:
# Creating dataFrame
import pandas as pd

dataFrame = pd.DataFrame(splitedContents)
dataFrame

In [0]:
# Create Histogram plot for class 'M' and class 'B'
import matplotlib.pyplot as plt

#tmp = dataframe for class B only
tmp = dataFrame.loc[dataFrame[1] == 'B',:]

#tmp2 = dataframe for class M only
tmp2 = dataFrame.loc[dataFrame[1] == 'M',:]

plt.hist(tmp)
plt.show()

In [0]:
# Create Histogram plot for column class only
import matplotlib.pyplot as plt
# extract column of the classes from dataframe
Class = dataFrame.loc[:,1]
Class

plt.hist(Class)
plt.show()

In [0]:
# remove first and second columns (instance id and class) 
Data = dataFrame.drop(columns=0)
Data = Data.drop(columns=1)
Data

In [0]:
# create scatter plot for the first attribute
import numpy

plt.figure(figsize=(52,2))
plt.scatter(dataFrame.loc[1:100,2],dataFrame.loc[1:100,1])
plt.show()

In [0]:
#create attribute list to create box plot
attributesList = []
i = 2
j = 0
while i < 32:
    listTemp = Data.loc[:,i].values.tolist()
    listTemp2 = [float(k) for k in listTemp]
    attributesList.insert(j , listTemp2)
    i = i + 1
    j = j + 1

attributesList

In [0]:
# Create boxplot
plt.figure(figsize=(20,25))
plt.boxplot(attributesList)
plt.show()

In [0]:
#Calculate correlation coefficient for Attribute List
import numpy
correlationMatrix = numpy.corrcoef(attributesList)
correlationMatrix

In [0]:
#Visualize correlationMatrix using imshow
plt.imshow(correlationMatrix)

In [0]:
# Zscore normalization
from scipy import stats

normalizedZscoreList = stats.zscore(attributesList)

print(normalizedZscoreList)

plt.hist(attributesList[0])

plt.show()

plt.hist(normalizedZscoreList[0])

plt.show()

plt.boxplot(attributesList[0])

plt.show()

plt.boxplot(normalizedZscoreList[0])

plt.show()

In [0]:
#transpose normalizedAtrributeList 
transposedNormalizedData = pd.DataFrame(normalizedZscoreList)
transposedNormalizedData = transposedNormalizedData.transpose()
normalizedDataList = transposedNormalizedData.values.tolist()
normalizedDataList

In [0]:
# Select number of components (feature projection)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(normalizedDataList)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
principalDf

In [0]:
#split data set to training data (70% of data, 398 instances) and test data (30% of data, 171 instances)
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=4, test_size=0.3,train_size=0.7, random_state=1)

#convert data and Class (dataframes) to arrays
dataArray = numpy.array(principalDf)
classArray = numpy.array(Class)

#sss.get_n_splits(dataArray, classArray)

for train_index, test_index in sss.split(dataArray, classArray):
#    print("TRAIN:", train_index, "TEST:", test_index)
    data_train, data_test = dataArray[train_index], dataArray[test_index]
    Class_train, Class_test = classArray[train_index], classArray[test_index]

print("-----------------------------")
print(len(data_train))
print("==========")
print(data_train)
print("-----------------------------")
print(len(data_test))
print("==========")
print(data_test)
print("-----------------------------")
print(len(Class_train))
print("==========")
print(Class_train)
print("-----------------------------")
print(len(Class_test))
print("==========")
print(Class_test)

In [0]:
#create K-neighbours model
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
KN_model = KNeighborsClassifier(n_neighbors=8)
# Train the model using the training sets
KN_model.fit(data_train,Class_train)
#Predict Output
KN_predicted= KN_model.predict(data_test) 

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Class_test, KN_predicted))
print(metrics.confusion_matrix(Class_test, KN_predicted))
print(metrics.classification_report(Class_test, KN_predicted,target_names=['M','B']))


In [0]:
#create SVM model
#Import svm model
from sklearn import svm
#Create a svm Classifier
SVM_model = svm.SVC(kernel='linear',C=1) # Linear Kernel
#Train the model using the training sets
SVM_model.fit(data_train, Class_train)
#Predict the response for test dataset
SVM_pred = SVM_model.predict(data_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Class_test, SVM_pred))
print(metrics.confusion_matrix(Class_test, SVM_pred))
print(metrics.classification_report(Class_test, SVM_pred,target_names=['M','B']))

In [0]:
#create SVC model

# Create a SVC classifier using an RBF kernel
SVC_model = svm.SVC(kernel='rbf', random_state=0, gamma=1, C=10)
# Train the classifier
SVC_model.fit(data_train, Class_train)# Visualize the decision boundaries
#Predict the response for test dataset
SVC_pred = SVC_model.predict(data_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Class_test, SVC_pred))
print(metrics.confusion_matrix(Class_test, SVC_pred))
print(metrics.classification_report(Class_test, SVC_pred,target_names=['M','B']))

In [0]:
#create logistic regression model
# import the class
from sklearn.linear_model import LogisticRegression
# instantiate the model (using the default parameters)
LogReg_model = LogisticRegression(C=10000)
# fit the model with data
LogReg_model.fit(data_train,Class_train)#
LogReg_pred=LogReg_model.predict(data_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Class_test, LogReg_pred))
print(metrics.confusion_matrix(Class_test, LogReg_pred))
print(metrics.classification_report(Class_test, LogReg_pred,target_names=['M','B']))

In [0]:
#apply GridSearchCV on each classfifier model to get best parameters
from sklearn.model_selection import GridSearchCV

KN_params = { 
        'n_neighbors' : [1,2,3,4,5,6,7,8]
}

SVM_params = { 
    'C': [1,10,100,1000,10000]
}

SVC_params = { 
    'C': [1,10,100,1000,10000],
    'gamma' : [0.01,0.1,1,10,100]
}

LogReg_params = { 
    'C': [1,10,100,1000,10000]
}

CV_KN = GridSearchCV(estimator=KN_model, param_grid=KN_params, cv= 10)
CV_KN.fit(data_train, Class_train)
print("KN_best_parameters",CV_KN.best_params_ )

CV_SVM = GridSearchCV(estimator=SVM_model, param_grid=SVM_params, cv= 10)
CV_SVM.fit(data_train, Class_train)
print("SVM_best_parameters",CV_SVM.best_params_ )

CV_SVC = GridSearchCV(estimator=SVC_model, param_grid=SVC_params, cv= 10)
CV_SVC.fit(data_train, Class_train)
print("SVC_best_parameters",CV_SVC.best_params_ )

CV_LogReg = GridSearchCV(estimator=LogReg_model, param_grid=LogReg_params, cv= 10)
CV_LogReg.fit(data_train, Class_train)
print("LogReg_best_parameters",CV_LogReg.best_params_ )